ECE 449 Lab D41

Submitted by: Priyanka Goradia

Student ID: 1669487


---



# **1. Data Ingestion**

**1.1 Importing necessary libraries**

In [ ]:
pip install tensorflow scikeras scikit-learn

In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 23.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import get_scorer_names
from sklearn.metrics import make_scorer, f1_score
from keras import layers
from keras_tuner.tuners import RandomSearch

1.2 Importing the Dataset

In [ ]:
dataFrame = pd.read_csv('/content/wine.csv', header = None)

In [ ]:
print(dataFrame)

     0      1     2     3     4    5     6     7     8     9      10    11  \
0     1  14.23  1.71  2.43  15.6  127  2.80  3.06  0.28  2.29   5.64  1.04   
1     1  13.20  1.78  2.14  11.2  100  2.65  2.76  0.26  1.28   4.38  1.05   
2     1  13.16  2.36  2.67  18.6  101  2.80  3.24  0.30  2.81   5.68  1.03   
3     1  14.37  1.95  2.50  16.8  113  3.85  3.49  0.24  2.18   7.80  0.86   
4     1  13.24  2.59  2.87  21.0  118  2.80  2.69  0.39  1.82   4.32  1.04   
..   ..    ...   ...   ...   ...  ...   ...   ...   ...   ...    ...   ...   
173   3  13.71  5.65  2.45  20.5   95  1.68  0.61  0.52  1.06   7.70  0.64   
174   3  13.40  3.91  2.48  23.0  102  1.80  0.75  0.43  1.41   7.30  0.70   
175   3  13.27  4.28  2.26  20.0  120  1.59  0.69  0.43  1.35  10.20  0.59   
176   3  13.17  2.59  2.37  20.0  120  1.65  0.68  0.53  1.46   9.30  0.60   
177   3  14.13  4.10  2.74  24.5   96  2.05  0.76  0.56  1.35   9.20  0.61   

       12    13  
0    3.92  1065  
1    3.40  1050  
2    3.17

# **2. Data Pre-Processing**

2.1 Data Labeling
- There is a need to label columns to the dataset.

In [ ]:
print(dataFrame.columns)

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')


In [ ]:
columns = [
'class', 'Alcohol', 'Malicacid', 'Ash', 'Alcalinity_of_ash', 'Magnesium','Total_phenols', 'Flavanoids','Nonflavanoid_phenols', 'Proanthocyanins','Color_intensity', 'Hue', '0D280_0D315_of_diluted_wines', 'Proline'
]
dataFrame.columns = columns
print(dataFrame.columns)

Index(['class', 'Alcohol', 'Malicacid', 'Ash', 'Alcalinity_of_ash',
       'Magnesium', 'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols',
       'Proanthocyanins', 'Color_intensity', 'Hue',
       '0D280_0D315_of_diluted_wines', 'Proline'],
      dtype='object')


In [ ]:
print(dataFrame)

     class  Alcohol  Malicacid   Ash  Alcalinity_of_ash  Magnesium  \
0        1    14.23       1.71  2.43               15.6        127   
1        1    13.20       1.78  2.14               11.2        100   
2        1    13.16       2.36  2.67               18.6        101   
3        1    14.37       1.95  2.50               16.8        113   
4        1    13.24       2.59  2.87               21.0        118   
..     ...      ...        ...   ...                ...        ...   
173      3    13.71       5.65  2.45               20.5         95   
174      3    13.40       3.91  2.48               23.0        102   
175      3    13.27       4.28  2.26               20.0        120   
176      3    13.17       2.59  2.37               20.0        120   
177      3    14.13       4.10  2.74               24.5         96   

     Total_phenols  Flavanoids  Nonflavanoid_phenols  Proanthocyanins  \
0             2.80        3.06                  0.28             2.29   
1            

2.2 Declaring a Preprocessor with:
- Data Imputation: Replacing NAN values with mean
- Encoding Categorical Variables: One Hot Encoding
Class 1: 100
Class 2: 010
Class 3: 001
-  Scaling: MinMax Scaling

In [ ]:
preprocessor = Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', MinMaxScaler())
                ])
x = dataFrame.drop('class',axis = 1)
x_pre_processed = preprocessor.fit_transform(x)

In [ ]:
encoder = OneHotEncoder()
encoded_y = encoder.fit_transform(dataFrame[['class']]).toarray()

In [ ]:
y_encoded_classes = pd.DataFrame(encoded_y, columns = ['Class1', 'Class2', 'Class3'])
x_pre_processed_data = pd.DataFrame(x_pre_processed, columns = columns [1:14])

In [ ]:
print(x_pre_processed_data)

      Alcohol  Malicacid       Ash  Alcalinity_of_ash  Magnesium  \
0    0.842105   0.191700  0.572193           0.257732   0.619565   
1    0.571053   0.205534  0.417112           0.030928   0.326087   
2    0.560526   0.320158  0.700535           0.412371   0.336957   
3    0.878947   0.239130  0.609626           0.319588   0.467391   
4    0.581579   0.365613  0.807487           0.536082   0.521739   
..        ...        ...       ...                ...        ...   
173  0.705263   0.970356  0.582888           0.510309   0.271739   
174  0.623684   0.626482  0.598930           0.639175   0.347826   
175  0.589474   0.699605  0.481283           0.484536   0.543478   
176  0.563158   0.365613  0.540107           0.484536   0.543478   
177  0.815789   0.664032  0.737968           0.716495   0.282609   

     Total_phenols  Flavanoids  Nonflavanoid_phenols  Proanthocyanins  \
0         0.627586    0.573840              0.283019         0.593060   
1         0.575862    0.510549       

In [ ]:
print(y_encoded_classes)

     Class1  Class2  Class3
0       1.0     0.0     0.0
1       1.0     0.0     0.0
2       1.0     0.0     0.0
3       1.0     0.0     0.0
4       1.0     0.0     0.0
..      ...     ...     ...
173     0.0     0.0     1.0
174     0.0     0.0     1.0
175     0.0     0.0     1.0
176     0.0     0.0     1.0
177     0.0     0.0     1.0

[178 rows x 3 columns]


# **3. Data Splitting**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_pre_processed_data, y_encoded_classes, test_size=0.1, stratify=y_encoded_classes, random_state=42)

# **4. Model Selection**

In [ ]:
input_dim = x_train.shape[1]

In [ ]:
def model_build(hyper):
  model = keras.Sequential()
  model.add(keras.layers.Input(shape=(input_dim,)))
  model.add(layers.Dense(units=hyper.Int('units', min_value=32, max_value=128,step=32),
                         activation='relu'))
  model.add(keras.layers.Dense(units=3, activation='softmax'))
  model.compile(optimizer=keras.optimizers.Adam(
                hyper.Choice('learning_rate',
                             values=[0.001, 0.01, 0.1])),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])
  return model

# **5. Parameter Exploration & Hyperparameter Tuning**

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
tuner = RandomSearch(
    model_build,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
)

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.01, 0.1], 'ordered': True}


In [ ]:
#works like model.fit
tuner.search(x_train,y_train,epochs=5, validation_data=(x_test,y_test), folds=stratified_kfold.split(x_train, y_train))

In [ ]:
tuner.results_summary(3)

Results summary
Results in ./untitled_project
Showing 3 best trials
Objective(name="val_loss", direction="min")

Trial 4 summary
Hyperparameters:
units: 128
learning_rate: 0.01
Score: 0.9444444179534912

Trial 2 summary
Hyperparameters:
units: 32
learning_rate: 0.1
Score: 0.9629629453023275

Trial 3 summary
Hyperparameters:
units: 96
learning_rate: 0.01
Score: 0.9629629453023275


# **6. Model Training**

From the hyperparameter tunining we can see that the best model is with the following hyperparameters and with the least validation error (val_loss):

Hyperparameters:
units: 128
learning_rate: 0.01
Score: 0.9444444179534912

In [ ]:
#Training with the best model hyperparameters that minimize the validation error (val_loss)
best_model = tuner.get_best_models(1)[0]
best_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10


5/5 [==============================] - 1s 85ms/step - loss: 0.4789 - accuracy: 0.8750 - val_loss: 0.4385 - val_accuracy: 0.8889
Epoch 2/10
5/5 [==============================] - 0s 18ms/step - loss: 0.3124 - accuracy: 0.9250 - val_loss: 0.3388 - val_accuracy: 0.9444
Epoch 3/10
5/5 [==============================] - 0s 15ms/step - loss: 0.2411 - accuracy: 0.9500 - val_loss: 0.2459 - val_accuracy: 0.9444
Epoch 4/10
5/5 [==============================] - 0s 19ms/step - loss: 0.1898 - accuracy: 0.9563 - val_loss: 0.2167 - val_accuracy: 0.9444
Epoch 5/10
5/5 [==============================] - 0s 15ms/step - loss: 0.1473 - accuracy: 0.9625 - val_loss: 0.1800 - val_accuracy: 0.9444
Epoch 6/10
5/5 [==============================] - 0s 19ms/step - loss: 0.1219 - accuracy: 0.9750 - val_loss: 0.1388 - val_accuracy: 0.9444
Epoch 7/10
5/5 [==============================] - 0s 15ms/step - loss: 0.1100 - accuracy: 0.9812 - val_loss: 0.1141 - val_accuracy: 0.9444
Epoch 8/10
5/5 [======================

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Add preprocessing steps as needed
    ('keras_model', best_model)  # Add the Keras model as a step
])

# **7. Performance Evaluation**

In [ ]:
#storing the predictions of the best model in y_pred
y_pred = best_model.predict(x_test)

#converting pandas dataframe to numpy array
y_test_np = y_test.to_numpy()
y_test_classes = np.argmax(y_test_np, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate the F1-score
f1_score_value = f1_score(y_test_classes, y_pred_classes, average='weighted')

print(f"F1 Score: {f1_score_value}")

1/1 [==============================] - 0s 31ms/step
F1 Score: 0.9448329448329448


**The F1 score of the best model:
F1 Score: 0.9448329448329448**